# Libraries

In [6]:
cd /home/tvangraft/tudelft/thesis/metaengineering

/home/tvangraft/tudelft/thesis/metaengineering


In [7]:
from src.pipeline.dataloader import DataLoader
from src.orchestrator.orchestrator import SklearnOrchestrator

from src.settings.strategy import Strategy
from src.settings.tier import Tier

from src.utils.utils import build_config

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor

In [8]:
def get_dl_config_for_strategy(tier: Tier):
    lookup = {
        Tier.TIER0: dict(
            additional_filters=["is_precursor",],
            additional_transforms=["log_fold_change_protein",]
        ),
        Tier.TIER1: dict(
            additional_frames=["interaction_frame",],
            additional_filters=[
                "is_precursor",
            ],
            additional_transforms=[
                "log_fold_change_protein",
                "ppi_coo_matrix",
            ]
        )
    }
    return lookup.get(tier)

In [9]:
params = {
    # 'SVR': {
    #     'regressor__regressor': SVR(),
    #     'regressor__regressor__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    #     'regressor__regressor__gamma': ['scale'],
    #     'regressor__regressor__C': [1, 10, 100, 1000],
    #     'regressor__preprocessor__num__scaler': [MinMaxScaler(), StandardScaler()],
    # },
    'RandomForestRegressor': {
        'regressor__regressor': RandomForestRegressor(),
        'regressor__regressor__n_estimators': [10, 25, 50, 75, 100],
        'regressor__regressor__criterion': ['squared_error', 'friedman_mse'],
        'regressor__regressor__max_depth': [5, 10, 20],
        'regressor__preprocessor__num__scaler': [MinMaxScaler(), StandardScaler(), RobustScaler()]
    },
    'ElasticNet': {
        'regressor__regressor': ElasticNet(),
        'regressor__regressor__l1_ratio': [0.01, 0.25, 0.5, 0.75, 1],
        'regressor__regressor__tol': [0.01],
        'regressor__preprocessor__num__scaler': [MinMaxScaler(), StandardScaler(), RobustScaler()]
    },
}

strategies = [Strategy.ALL, Strategy.ONE_VS_ALL, Strategy.METABOLITE_CENTRIC]
tiers = [Tier.TIER1]

In [ ]:
DataLoader.DATA_FOLDER = './data/training/'
for tier in tiers:
    for strategy in strategies:
        orchestrator: SklearnOrchestrator = SklearnOrchestrator()
        orchestrator.prepare_orchestrator(
            *build_config(
                strategy=strategy,
                tier=tier,
                params=params,
                forced_training=True,
                forced_testing=True,
                **get_dl_config_for_strategy(tier)
            )
        )
        orchestrator.run()